# Configuring `Output Parsers` to customize output

- We can configure the output parsers to customize the output of the chatbot.
- Return output as simple string for printing, or as a list / object for complex parsing downstream in multimodal agents / other applications

## Import dependencies

In [1]:
var Bedrock = require('@langchain/community/llms/bedrock').Bedrock;
var ChatPromptTemplate = require('@langchain/core/prompts').ChatPromptTemplate;
var StringOutputParser = require('@langchain/core/output_parsers').StringOutputParser;
var CommaSeparatedListOutputParser = require('@langchain/core/output_parsers').CommaSeparatedListOutputParser;
var StructuredOutputParser = require('langchain/output_parsers').StructuredOutputParser;

## Instantiate the `model` client

In [13]:
var model = new Bedrock({
    model_id:'amazon.titan-text-express-v1',
    temperature: 0,
    maxTokenCount: 1024,
    topP: 0.9,
    verbose: true
});

## Create prompt template (string/list)

In [ ]:
var promptMessages = ChatPromptTemplate.fromTemplate(
    "Provide 5 synonyms, separated by commas, for the following word {input}"
);
// var promptMessages = prompts.ChatPromptTemplate.fromMessages([
//     ["system", "Output only 5 words which are synonyms, separated by commas, with no other words for the following word {input}."],
//     ["user", "{input}"]
// ]);

In [ ]:
var input = "dog";

## Create output parser (string)

In [ ]:
var outputParserString = new StringOutputParser();

## Create a chain (string)

- We send the prompt to the model (pipe it)

In [ ]:
var chainOutputString = promptMessages.pipe(model).pipe(outputParserString);

## Invoke LLM

- Given input we invoke LLM and get 5 synonyms using output parser `StringOutputParser`

In [ ]:
chainOutputString.invoke({input}).then((output) => console.log(output));

## Create output parser (string list)

In [ ]:
var outputParserStringList = new CommaSeparatedListOutputParser();

## Create a chain (string list)

- We send the prompt to the model (pipe it), and the model to the output parser

In [ ]:
var chainOutputStringList = promptMessages.pipe(model).pipe(outputParserStringList);

## Invoke LLM

- Given input we invoke LLM and get 5 synonyms using output parser `CommaSeparatedListOutputParser`

In [ ]:
chainOutputStringList.invoke({input}).then((output) => console.log(output));

## Create output parser (structured, basic, 1 level)

- Given a well known structure, we instruct the LLM to extract from given `input` values that would fit the specified description.

In [ ]:
var outputParserStructuredBasic = StructuredOutputParser.fromNamesAndDescriptions({
    name: 'the name of the person',
    age: 'the age of the person'
})

## Formatted instruction output

In [ ]:
console.log(outputParserStructuredBasic.getFormatInstructions());

## Create prompt template (structured)

- We need to configure template to accept formatting instructions as well as a new imperative system message.

In [7]:
var promptStructured = ChatPromptTemplate.fromTemplate(`
    Extract information from the following phrase.
    Formatting instructions: {format_instructions}
    Phrase: {input}
`);

## Create a chain (structured, basic)

- We send the prompt to the model (pipe it), and the model to the output parser

In [ ]:
var chainOutputStructured = promptStructured.pipe(model).pipe(outputParserStructuredBasic);

## The new input

- For the structured basic output parser

In [ ]:
var input = 'Max is 30 years old.';

## Invoke LLM

- Given input we invoke LLM and get object containing `{name:<value>, age:<value>}`

In [ ]:
chainOutputStructured.invoke({
    input,
    format_instructions: outputParserStructured.getFormatInstructions()
}).then((output) => {
    console.log(output);
    console.log(typeof output);
});

## Create output parser (structured, advanced, multilevel)

- Given a well known structure, we instruct the LLM to extract from given `input` values that would fit the specified description.
- We need to install `zod`

```shell
yarn add zod
```

## Import dependencies

In [3]:
var z = require('zod').z;

In [4]:
var outputParserStructuredAdvanced = StructuredOutputParser.fromZodSchema(
    z.object({
        recipe: z.string().describe("the name of the recipe"),
        ingredients: z.array(z.string()).describe("ingredients"),
    })
);

## Formatted instruction output

In [5]:
console.log(outputParserStructuredAdvanced.getFormatInstructions());

You must format your output as a JSON value that adheres to a given "JSON Schema" instance.

"JSON Schema" is a declarative language that allows you to annotate and validate JSON documents.

For example, the example "JSON Schema" instance {{"properties": {{"foo": {{"description": "a list of test words", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}}}
would match an object with one required property, "foo". The "type" property specifies "foo" must be an "array", and the "description" property semantically describes it as "a list of test words". The items within "foo" must be strings.
Thus, the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of this example "JSON Schema". The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Your output will be parsed and type-checked according to the provided schema instance, so make sure all fields in your output match the schema exactly and there are no trailing commas!

Here is the JS

## Create a chain (structured, advanced)

- We send the prompt to the model (pipe it), and the model to the output parser

In [8]:
var chainOutputStructuredAdvanced = promptStructured.pipe(model).pipe(outputParserStructuredAdvanced);

## The new input

- For the structured advanced output parser

In [9]:
var input = "The ingredients for a Spaghetti Bolognese recipe are tomatoes, minced beef, garlic and pasta.";

## Invoke LLM

- Given input we invoke LLM and get object containing `{recipe:<value>, ingredients:[<value>]}`

In [ ]:
chainOutputStructuredAdvanced.invoke({
    input,
    format_instructions: outputParserStructuredAdvanced.getFormatInstructions()
}).then((output) => {
    console.log(output);
    console.log(typeof output);
});

Promise { <pending> }

[llm/start] [1:llm:Bedrock] Entering LLM run with input: {
  "prompts": [
    "Human: \n    Extract information from the following phrase.\n    Formatting instructions: You must format your output as a JSON value that adheres to a given \"JSON Schema\" instance.\n\n\"JSON Schema\" is a declarative language that allows you to annotate and validate JSON documents.\n\nFor example, the example \"JSON Schema\" instance {{\"properties\": {{\"foo\": {{\"description\": \"a list of test words\", \"type\": \"array\", \"items\": {{\"type\": \"string\"}}}}}}, \"required\": [\"foo\"]}}}}\nwould match an object with one required property, \"foo\". The \"type\" property specifies \"foo\" must be an \"array\", and the \"description\" property semantically describes it as \"a list of test words\". The items within \"foo\" must be strings.\nThus, the object {{\"foo\": [\"bar\", \"baz\"]}} is a well-formatted instance of this example \"JSON Schema\". The object {{\"properties\": {{\"foo\": [\"bar\", \"ba

Error: Failed to parse. Text: "
```json
{
    "recipe": "Spaghetti Bolognese",
    "ingredients": [
        "tomatoes",
        "minced beef",
        "garlic",
        "pasta"
    ]
". Error: SyntaxError: Expected ',' or '}' after property value in JSON at position 143
    at StructuredOutputParser.parse (/workspace/node_modules/langchain/dist/output_parsers/structured.cjs:83:19)
    at StructuredOutputParser.parseResult (/workspace/node_modules/@langchain/core/dist/output_parsers/base.cjs:60:21)
    at StructuredOutputParser._callWithConfig (/workspace/node_modules/@langchain/core/dist/output_parsers/base.cjs:42:72)
    at StructuredOutputParser._callWithConfig (/workspace/node_modules/@langchain/core/dist/runnables/base.cjs:214:33)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async RunnableSequence.invoke (/workspace/node_modules/@langchain/core/dist/runnables/base.cjs:1056:27)